<font size="6">**Datathon- Auto ML**</font>



In [35]:
#import libararies
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

import plotly.express as px
from plotly.offline import init_notebook_mode, iplot, plot
import plotly as py
init_notebook_mode(connected = True)
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly import tools

import optuna  
#from verstack import LGBMTuner

import model_func
#to make sure kernel restart is not needed after making changes in .py
import importlib
importlib.reload(model_func)

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder

import sklearn.metrics as metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score,roc_curve, roc_auc_score, auc


from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector

from datetime import datetime


# Machine Learnig Classification Models - AutoML

In [36]:
df =  pd.read_excel('01_data_compiled_selected_patients_4221.xlsx')
print(df.shape)

(4221, 84)


In [37]:
#display all columns
pd.set_option("display.max_columns",None)
df.head()

,caseid,complications,icu_days,death_inhosp,pre_post_cr,age,sex,height,weight,bmi,asa,optype,approach,emop,ane_type,preop_htn,preop_dm,preop_pft,optype_00,optype_01,optype_02,optype_03,optype_04,optype_05,optype_06,optype_07,optype_08,optype_09,optype_10,approach_00,approach_01,approach_02,ane_type_00,ane_type_01,ane_type_02,preop_pft_00,preop_pft_01,preop_pft_02,preop_pft_03,preop_pft_04,preop_pft_05,preop_pft_06,preop_pft_07,preop_pft_08,asa_00,asa_01,asa_02,asa_03,asa_04,asa_05,b4_alb,b4_alt,b4_ammo,b4_aptt,b4_ast,b4_be,b4_bun,b4_ccr,b4_cl,b4_cr,b4_crp,b4_esr,b4_fib,b4_gfr,b4_gluc,b4_hb,b4_hco3,b4_hct,b4_ica,b4_k,b4_lac,b4_na,b4_p,b4_pco2,b4_ph,b4_plt,b4_po2,b4_pt%,b4_ptinr,b4_ptsec,b4_sao2,b4_tbil,b4_tprot,b4_wbc
0,1,0,0,0,0.878049,77.0,0,160.2,67.50,26.3,cat02,Colorectal,Open,0,General,1,0,Normal,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,3.4,20.0,NaN,28.0,20.0,NaN,15.0,NaN,100.0,0.88,22.41,NaN,254.0,83.8,198.0,13.4,NaN,38.9,NaN,3.1,NaN,138.0,NaN,NaN,NaN,146.0,NaN,89.0,1.08,12.3,NaN,0.8,6.2,15.16
1,2,0,0,0,0.953488,54.0,0,167.3,54.80,19.6,cat02,Stomach,Open,0,General,0,0,Normal,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,3.1,19.0,NaN,28.6,29.0,NaN,12.0,NaN,115.0,1.02,16.18,NaN,290.0,75.9,118.0,10.2,NaN,32.5,NaN,4.7,NaN,143.0,NaN,NaN,NaN,251.0,NaN,104.0,0.97,11.1,NaN,0.5,5.2,4.36
2,7,1,3,0,NaN,52.0,1,167.7,62.30,22.2,cat02,Major resection,Videoscopic,0,General,0,0,Normal,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,4.4,16.0,NaN,26.4,20.0,0.8,18.0,NaN,106.0,0.84,0.09,NaN,271.0,70.9,94.0,11.8,25.4,35.1,1.17,4.4,1.2,139.0,NaN,47.0,7.34,229.0,154.0,109.0,0.95,10.8,99.0,0.4,7.4,4.28
3,9,0,0,0,NaN,32.0,1,157.9,50.90,20.4,cat01,Biliary/Pancreas,Videoscopic,0,General,0,0,Normal,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,4.4,8.0,NaN,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.8,NaN,39.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,133.0,NaN,NaN,NaN,NaN,NaN,0.8,7.2,11.30
4,10,0,1,0,0.725000,72.0,0,162.5,62.75,23.8,cat03,Stomach,Videoscopic,0,General,1,0,Normal,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,3.2,54.0,NaN,27.3,57.0,NaN,18.0,NaN,104.0,1.15,0.10,NaN,336.0,62.5,315.0,13.6,NaN,40.3,NaN,5.4,NaN,139.0,3.5,NaN,NaN,195.0,NaN,93.0,1.04,11.9,NaN,0.9,5.6,13.38


## Feature Engineering 

In [38]:
df['complications'].value_counts()

0    3957
1     264
Name: complications, dtype: int64

In [39]:
#dropping not required and categorical features
df= df.drop(['death_inhosp','caseid','icu_days','height','weight','pre_post_cr','approach',\
             'ane_type','asa', 'optype','preop_pft'],axis= 1)
#df = df.drop(df.filter(regex='b4').columns, axis=1)
#df = df.drop(df.filter(regex='optype_').columns, axis=1)
#df = df.drop(df.filter(regex='preop_pft_').columns, axis=1)

## Train - Test Split

In [40]:
# X - all the feature columns except target variable
X= df.drop(['complications'],axis= 1)
# y - target variable
y= df['complications']

#80% training data set and 20% validation set
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42)
print("%r, %r, %r" % (X.shape, X_train.shape, X_test.shape))

(4221, 72), (3376, 72), (845, 72)


# AUTOML

## tpot

In [41]:
import tpot
from tpot import TPOTClassifier

In [ ]:
%%time
tpot = TPOTClassifier(generations=5, population_size=30,
                          offspring_size=None,
                          scoring='roc_auc',
                          max_time_mins=None, 
                          max_eval_time_mins=5,
                          random_state=101,
                          warm_start=True,
                          early_stop=20,
                          verbosity=2)
tpot.fit(X_train, y_train)

Imputing missing values in feature set


Optimization Progress:   0%|          | 0/180 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.8738662943777019


In [43]:
tpot.score(X_test, y_test)

Imputing missing values in feature set


0.9067085953878409

In [44]:
print(classification_report(y_test, tpot.predict(X_test)))

Imputing missing values in feature set
              precision    recall  f1-score   support

           0       0.95      1.00      0.97       792
           1       0.92      0.23      0.36        53

    accuracy                           0.95       845
   macro avg       0.94      0.61      0.67       845
weighted avg       0.95      0.95      0.94       845



In [45]:
confusion_matrix = metrics.confusion_matrix(y_test, tpot.predict(X_test))
confusion_matrix

Imputing missing values in feature set


array([[791,   1],
       [ 41,  12]], dtype=int64)

# Appendix

## Auto-Sklearn

In [ ]:
pip install auto-sklearn

In [33]:
import autosklearn.classification 
# configure auto-sklearn
automl = autosklearn.classification.AutoSklearnClassifier(
                                    time_left_for_this_task=30, 
                                    per_run_time_limit=10)

ModuleNotFoundError: No module named 'autosklearn'

In [ ]:
automl.fit(X_test, y_train)

In [ ]:
# evaluate
pred = automl.predict(X_test)
print(classification_report(y_test, pred))

In [ ]:
confusion_matrix = metrics.confusion_matrix(y_test, y_pred)
confusion_matrix

## MLBox

## H2O